Wenb Scrapping Project

In [ ]:
#import required libraries
import requests
from bs4 import BeautifulSoup as bs


In [ ]:
url = 'http://books.toscrape.com/'
response = requests.get(url)

In [ ]:
response

<Response [200]>

In [ ]:
print(response.text[:1000])

<!DOCTYPE html>
<!--[if lt IE 7]>      <html lang="en-us" class="no-js lt-ie9 lt-ie8 lt-ie7"> <![endif]-->
<!--[if IE 7]>         <html lang="en-us" class="no-js lt-ie9 lt-ie8"> <![endif]-->
<!--[if IE 8]>         <html lang="en-us" class="no-js lt-ie9"> <![endif]-->
<!--[if gt IE 8]><!--> <html lang="en-us" class="no-js"> <!--<![endif]-->
    <head>
        <title>
    All products | Books to Scrape - Sandbox
</title>

        <meta http-equiv="content-type" content="text/html; charset=UTF-8" />
        <meta name="created" content="24th Jun 2016 09:29" />
        <meta name="description" content="" />
        <meta name="viewport" content="width=device-width" />
        <meta name="robots" content="NOARCHIVE,NOCACHE" />

        <!-- Le HTML5 shim, for IE6-8 support of HTML elements -->
        <!--[if lt IE 9]>
        <script src="//html5shim.googlecode.com/svn/trunk/html5.js"></script>
        <![endif]-->

        
            <link rel="shortcut icon" href="static/oscar/favicon.

In [ ]:
type(response.text)

str

In [ ]:
soup = bs(response.text)

In [ ]:
type(soup)

bs4.BeautifulSoup

In [ ]:
soup.find('title').text.strip()

'All products | Books to Scrape - Sandbox'

In [ ]:
books_tag = soup.find_all('article',class_='product_pod')

In [ ]:
len(books_tag)

20

In [ ]:
book_tag = books_tag[0]
book_tag

<article class="product_pod">
<div class="image_container">
<a href="catalogue/a-light-in-the-attic_1000/index.html"><img alt="A Light in the Attic" class="thumbnail" src="media/cache/2c/da/2cdad67c44b002e7ead0cc35693c0e8b.jpg"/></a>
</div>
<p class="star-rating Three">
<i class="icon-star"></i>
<i class="icon-star"></i>
<i class="icon-star"></i>
<i class="icon-star"></i>
<i class="icon-star"></i>
</p>
<h3><a href="catalogue/a-light-in-the-attic_1000/index.html" title="A Light in the Attic">A Light in the ...</a></h3>
<div class="product_price">
<p class="price_color">Â£51.77</p>
<p class="instock availability">
<i class="icon-ok"></i>
    
        In stock
    
</p>
<form>
<button class="btn btn-primary btn-block" data-loading-text="Adding..." type="submit">Add to basket</button>
</form>
</div>
</article>

In [ ]:
title_tag = book_tag.find('a',title=True)

In [ ]:
title_tag

<a href="catalogue/a-light-in-the-attic_1000/index.html" title="A Light in the Attic">A Light in the ...</a>

In [ ]:
title_tag.text

'A Light in the ...'

In [ ]:
title_tag['title']

'A Light in the Attic'

In [ ]:
## Retrieving Full Name of title
title = book_tag.find('a', title=True)['title']
title

'A Light in the Attic'

In [ ]:
## For Rating
rating = book_tag.find('p')['class'][1]
rating

'Three'

In [ ]:
## Mentioned Price
#Here if we change the price in the websit backend it will not reflect here it will return the old value which is originally there in the website.
price = book_tag.find('p', class_='price_color').text[1:]
price

'£51.77'

In [ ]:
## Link of the book
link = 'http://books.toscrape.com/' + book_tag.find('a')['href']
link

'http://books.toscrape.com/catalogue/a-light-in-the-attic_1000/index.html'

This is using the function insted of retrieving values in single single way we can do ti at a time with a function

In [ ]:
def get_the_details(book_tag):
    title = book_tag.find('a', title=True)['title']
    rating = book_tag.find('p')['class'][1]
    price = book_tag.find('p', class_='price_color').text[1:]
    link = 'http://books.toscrape.com/' + book_tag.find('a')['href']
    return title, rating, price, link

get_the_details function takes a 'book_tag', extracts all the details from it and returns them.This will work when we call it.

Lets write some more functions to

In [ ]:
def get_soup(url):
    """Takes URL and returns a soup object"""
    resp = requests.get(url)
    if resp.status_code == 200:#Here we wrote 200 because it means Ok .
        return bs(resp.text)
    else: return None


def get_books(url):
    """Extact details from all the book tags"""
    soup = get_soup(url)
    book_tags = soup.find_all('article', class_='product_pod')

    books = []
    for book_tag in book_tags:
        books.append(get_the_details(book_tag))

    return books

In [ ]:
url = 'http://books.toscrape.com/'
books = get_books(url)
len(books)

20

In [ ]:
books[:3]

[('A Light in the Attic',
  'Three',
  '£51.77',
  'http://books.toscrape.com/catalogue/a-light-in-the-attic_1000/index.html'),
 ('Tipping the Velvet',
  'One',
  '£53.74',
  'http://books.toscrape.com/catalogue/tipping-the-velvet_999/index.html'),
 ('Soumission',
  'One',
  '£50.10',
  'http://books.toscrape.com/catalogue/soumission_998/index.html')]

Here is our another function

In [ ]:
import pandas as pd

def get_all_books(page = 3):
    books = []
    for i in range(1, page+1):
        ## This is how the url changes with every page
        url = f'http://books.toscrape.com/catalogue/page-{i}.html'
        soup = get_soup(url)
        if soup:
            book_tags = soup.find_all('article', class_='product_pod')

            for book_tag in book_tags:
                books.append(get_the_details(book_tag))

    books = pd.DataFrame(books, columns=['title', 'rating', 'price', 'link'])
    return books

We will only scrape first 3 pages to test our code


In [ ]:
df = get_all_books(3)
df.head()

,title,rating,price,link
0,A Light in the Attic,Three,£51.77,http://books.toscrape.com/a-light-in-the-attic...
1,Tipping the Velvet,One,£53.74,http://books.toscrape.com/tipping-the-velvet_9...
2,Soumission,One,£50.10,http://books.toscrape.com/soumission_998/index...
3,Sharp Objects,Four,£47.82,http://books.toscrape.com/sharp-objects_997/in...
4,Sapiens: A Brief History of Humankind,Five,£54.23,http://books.toscrape.com/sapiens-a-brief-hist...


In [ ]:
df.shape

(60, 4)

In [ ]:
import time
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as bs

def get_soup(url):
    """Takes URL and returns a soup object"""
    try:
        resp = requests.get(url)
    except:
        return None

    if resp.status_code == 200:
        return bs(resp.text)
    else:
        return None

#Here we used try and except to handle exceptions.
def get_the_details(book_tag):
    ## title
    try:
        title = book_tag.find('a', title=True)['title']
    except:
        title = None

    ## rating
    try:
        rating = book_tag.find('p')['class'][1]
    except:
        rating = None

    ## Price
    try:
        price = book_tag.find('p', class_='price_color').text[1:]
    except:
        price = None

    ## Link
    try:
        link = 'http://books.toscrape.com/' + book_tag.find('a')['href']
    except:
        price = None

    return title, rating, price, link



def get_all_books(page = 3):
    books = []
    for i in range(1, page+1):
        url = f'http://books.toscrape.com/catalogue/page-{i}.html'
        soup = get_soup(url)
        if soup:
            try:
                book_tags = soup.find_all('article', class_='product_pod')

                for book_tag in book_tags:
                    books.append(get_the_details(book_tag))
            except:
                print(f'Error reading page {i} . . .')

            time.sleep(5) # sleep before making the next request

    books = pd.DataFrame(books, columns=['title', 'rating', 'price', 'link'])
    return books

In [ ]:
#here we had passed 50 avlue as parameter.
df = get_all_books(50) # 20 books x 50 pages = 1000 books
df.head()

,title,rating,price,link
0,A Light in the Attic,Three,£51.77,http://books.toscrape.com/a-light-in-the-attic...
1,Tipping the Velvet,One,£53.74,http://books.toscrape.com/tipping-the-velvet_9...
2,Soumission,One,£50.10,http://books.toscrape.com/soumission_998/index...
3,Sharp Objects,Four,£47.82,http://books.toscrape.com/sharp-objects_997/in...
4,Sapiens: A Brief History of Humankind,Five,£54.23,http://books.toscrape.com/sapiens-a-brief-hist...


In [ ]:
df.shape

(1000, 4)

In [ ]:
df.to_csv('books.csv', index=False)

Here is our csv file

In [ ]:
df.tail()

,title,rating,price,link
55,The Torch Is Passed: A Harding Family Story,One,£19.09,http://books.toscrape.com/the-torch-is-passed-...
56,The Secret of Dreadwillow Carse,One,£56.13,http://books.toscrape.com/the-secret-of-dreadw...
57,The Pioneer Woman Cooks: Dinnertime: Comfort C...,One,£56.41,http://books.toscrape.com/the-pioneer-woman-co...
58,The Past Never Ends,Four,£56.50,http://books.toscrape.com/the-past-never-ends_...
59,The Natural History of Us (The Fine Art of Pre...,Three,£45.22,http://books.toscrape.com/the-natural-history-...
